## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

### Data Load

In [2]:
# 데이터 로드
IVF_train = pd.read_csv('../data/IVF_train_dataset_31.csv')
IVF_test = pd.read_csv('../data/IVF_test_dataset_31.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
IVF_X = IVF_train.drop(['임신_성공_여부', 'ID'], axis=1)
IVF_y = IVF_train['임신_성공_여부']

In [4]:
print(f"IVF_X shape: {IVF_X.shape}")
print(f"IVF_test shape: {IVF_test.drop('ID', axis=1).shape}")

IVF_X shape: (250052, 97)
IVF_test shape: (87891, 97)


### 인코딩 

In [5]:
IVF_categorical_columns = [
    "시술_당시_나이",
    "배란_유도_유형",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [6]:
# 모든 범주형 변수를 문자열로 변환
IVF_X[IVF_categorical_columns] = IVF_X[IVF_categorical_columns].astype(str)
IVF_test[IVF_categorical_columns] = IVF_test[IVF_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
IVF_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

IVF_X[IVF_categorical_columns] = IVF_encoder.fit_transform(IVF_X[IVF_categorical_columns])
IVF_test[IVF_categorical_columns] = IVF_encoder.transform(IVF_test[IVF_categorical_columns])

## Modeling

In [7]:
# 데이터 분할
IVF_X_train, IVF_X_test, IVF_y_train, IVF_y_test = train_test_split(IVF_X, IVF_y, test_size=0.2, random_state=42)

### IVF 데이터

In [8]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 8000),
        'max_depth': trial.suggest_int('max_depth', 3, 300),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 2048),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        
        'boosting_type' : 'gbdt',
        'random_state': 42,
        'objective': 'binary',
        'metric': 'auc',
        'verbose': -1,   
        'njobs': -1
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(IVF_X_train, IVF_y_train)
    
    y_pred_proba = model.predict_proba(IVF_X_test)[:, 1]
    
    auc = roc_auc_score(IVF_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=800)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-16 01:12:43,560] A new study created in memory with name: no-name-f499162b-e5e8-45b7-a93e-00fc0f601e90
[I 2025-02-16 01:14:24,528] Trial 0 finished with value: 0.6879913149084951 and parameters: {'n_estimators': 3363, 'max_depth': 226, 'learning_rate': 0.07270647222625722, 'num_leaves': 467, 'min_child_samples': 79, 'subsample': 0.7167491809922264, 'reg_alpha': 0.056419440094137914, 'reg_lambda': 1.0282080389413704}. Best is trial 0 with value: 0.6879913149084951.
[I 2025-02-16 01:17:46,768] Trial 1 finished with value: 0.7080625420999695 and parameters: {'n_estimators': 2163, 'max_depth': 62, 'learning_rate': 0.007964985644536529, 'num_leaves': 1888, 'min_child_samples': 33, 'subsample': 0.6437056797996589, 'reg_alpha': 0.024579170827883943, 'reg_lambda': 0.012520027380750018}. Best is trial 1 with value: 0.7080625420999695.
[I 2025-02-16 01:18:30,862] Trial 2 finished with value: 0.732115292314218 and parameters: {'n_estimators': 1682, 'max_depth': 81, 'learning_rate': 0.0

KeyboardInterrupt: 

.